In [13]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from datetime import datetime, timedelta

In [14]:
data = pd.read_csv("/home/dev/Documents/Input/ALL CDN Data.csv")

In [15]:
data["Community"].unique()

array(['mimico', 'humber summit', 'sandringham-wellington',
       'mississauga valleys', 'appleby', 'bendale', 'dorset park',
       'cornell', 'pleasant view', 'bayview village',
       'north st. james town', 'little portugal', 'church-yonge corridor',
       'willowdale east', 'annex', 'unionville',
       'agincourt south-malvern west', 'thornhill', 'hurontario',
       'kensington-chinatown', 'bay street corridor', 'lasalle',
       'steeles', 'erindale', 'markland wood', 'queen street corridor',
       'central erin mills', 'city centre', 'east woodbridge',
       'broadview north', 'westminster-branson',
       'waterfront communities c1', 'mount pleasant west', 'moss park',
       'islington-city centre west', 'south richvale',
       'banbury-don mills', 'regent park', 'niagara', 'hillcrest village',
       'henry farm', 'old oakville', 'rockcliffe-smythe',
       "fletcher's creek south", 'milliken mills west', 'malvern',
       'casa loma', 'flemingdon park', 'forest hill s

In [16]:
data = data[data["Community"] == "port credit"]
data = data.dropna()
len(data)

131

In [17]:
data.dtypes

Address                           object
Approx_Square_Footage             object
Area                              object
Community                         object
Last_Status                       object
ML_Number                         object
Municipality                      object
Municipality_District             object
Sold_Price                       float64
Washrooms                          int64
lat                              float64
lng                              float64
calculated_Approx_Age             object
_id_y                             object
Current_Month                     object
Current_Price                    float64
HPI_Index_Current_Month          float64
HPI_Index_Sold_Month             float64
Sold_Month                        object
Bedrooms                         float64
Bedrooms_                        float64
Modified_Balcony                  object
Adjusted_Total_Parking_Spaces    float64
Parking_Category                  object
FLOOR_NUMBER    

In [18]:
data.columns

Index(['Address', 'Approx_Square_Footage', 'Area', 'Community', 'Last_Status',
       'ML_Number', 'Municipality', 'Municipality_District', 'Sold_Price',
       'Washrooms', 'lat', 'lng', 'calculated_Approx_Age', '_id_y',
       'Current_Month', 'Current_Price', 'HPI_Index_Current_Month',
       'HPI_Index_Sold_Month', 'Sold_Month', 'Bedrooms', 'Bedrooms_',
       'Modified_Balcony', 'Adjusted_Total_Parking_Spaces', 'Parking_Category',
       'FLOOR_NUMBER', 'latest_tax'],
      dtype='object')

In [19]:
columns = [
    "lat",
    "lng",
    "calculated_Approx_Age",
    "latest_tax",
    "Washrooms",
    "Bedrooms",
    "Bedrooms_",
    "Adjusted_Total_Parking_Spaces",
    "Parking_Category",
    "Modified_Balcony",
    "Approx_Square_Footage",
    "FLOOR_NUMBER",
    "Current_Price",
    "Address",
    "Sold_Month",
]
categorical_columns = [
    "Washrooms",
    "Bedrooms",
    "Adjusted_Total_Parking_Spaces",
    "Parking_Category",
    "Modified_Balcony",
    "FLOOR_NUMBER",
    "calculated_Approx_Age",
    "Approx_Square_Footage",
    # "Address",
    "Bedrooms_",
]
numerical_columns = [
    "latest_tax",
]

In [20]:
df = data[columns]

In [23]:
df['Sold_Month']

786      Aug-23
964      Aug-23
1591     Sep-23
1783     Sep-23
2499     Sep-23
          ...  
52633    Dec-23
53686    Jan-24
53899    Jan-24
54157    Jan-24
54397    Feb-24
Name: Sold_Month, Length: 131, dtype: object

In [24]:
from datetime import datetime, timedelta

df["Sold_Month"] = pd.to_datetime(df["Sold_Month"], format='%b-%y')

# Filter out rows with NaT (parsing errors)
df = df.dropna(subset=["Sold_Month"])

df["Sold_Month"]

/tmp/ipykernel_58988/3716845610.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sold_Month"] = pd.to_datetime(df["Sold_Month"], format='%b-%y')


786     2023-08-01
964     2023-08-01
1591    2023-09-01
1783    2023-09-01
2499    2023-09-01
           ...    
52633   2023-12-01
53686   2024-01-01
53899   2024-01-01
54157   2024-01-01
54397   2024-02-01
Name: Sold_Month, Length: 131, dtype: datetime64[ns]

In [33]:
current_date = datetime(2023, 12, 1).date()
start_date = current_date - timedelta(days=delta)
start_date

datetime.date(2023, 11, 1)

In [38]:
timedelta_values = [30, 60, 90, 120, 180]

# Flag to indicate if any data is found
data_found = False
# current_date = datetime.now().date()
current_date = pd.to_datetime(datetime(2023, 12, 1).date())
for delta in timedelta_values:
    # Calculate start_date based on the current delta
    start_date = pd.to_datetime(current_date - timedelta(days=delta))

    # Filter data for sold months within the specified time range
    filtered_df = df[
        (df["Sold_Month"] >= start_date) & (df["Sold_Month"] <= current_date)
    ]
    filtered_df = filtered_df.drop(["Address", "Sold_Month"], axis=1)

    print(len(filtered_df))
    # Check if filtered_df is not empty
    if len(filtered_df) != 0:
        # Select relevant columns for modeling
        X = filtered_df

        # Fit the nearest neighbors model
        knn = NearestNeighbors(n_neighbors=5)
        knn.fit(X)

        # Set the data_found flag to True
        data_found = True

        # Optionally, you can break out of the loop if you only want to use the first non-empty result
        # break
    else:
        pass

# Check if any data is found
if data_found:
    # Transform the query house data
    query_data = pd.DataFrame(query_house, index=[0])

    # Find the nearest neighbors for the query house
    distances, indices = knn.kneighbors(query_data)

    # Display the nearest neighbors
    nearest_neighbors = filtered_df.iloc[indices[0]]
    print(nearest_neighbors)
else:
    print("No samples found within the specified time ranges.")

10


ValueError: could not convert string to float: '15-Nov'

In [29]:
# Filter data for sold months within the last 30 days
current_date = datetime.now().date()
start_date = current_date - timedelta(days=30)
filtered_df = df[(df["Sold_Month"] >= start_date) & (df["Sold_Month"] <= current_date)]

TypeError: Invalid comparison between dtype=datetime64[ns] and date

In [ ]:
filtered_df

In [30]:
# Select relevant columns for modeling
X = filtered_df.drop(["Address", "Sold_Month"], axis=1)

# Fit the nearest neighbors model
knn = NearestNeighbors(
    n_neighbors=5
)  # You can adjust the number of neighbors as needed
knn.fit(X)

# Transform the query house data
query_data = pd.DataFrame(query_house, index=[0])

# Find the nearest neighbors for the query house
distances, indices = knn.kneighbors(query_data)

# Display the nearest neighbors
nearest_neighbors = filtered_df.iloc[indices[0]]
print(nearest_neighbors)

ValueError: could not convert string to float: '15-Nov'

In [ ]:
df = data[columns]

In [ ]:
# Preprocess the data
# Convert categorical features to dummy variables
df = pd.get_dummies(df, columns=categorical_columns, dtype=int)
df

In [ ]:
# Normalize numerical features
df = (df - df.min()) / (df.max() - df.min())
df

In [ ]:
df.describe()

In [ ]:
# # Define the similarity metric
# def similarity(house1, house2):
#     # Calculate the cosine similarity between two houses
#     return cosine_similarity(
#         house1.values.reshape(1, -1), house2.values.reshape(1, -1)
#     )[0][0]


# # Define the recommendation function
# def recommend(query_house, n=10):
#     # Initialize an empty list to store the similarity scores
#     scores = []

#     # Loop through all the houses in the dataset
#     for i in range(len(df)):
#         # Get the current house
#         house = df.iloc[i]

#         # Calculate the similarity score with the query house
#         score = similarity(query_house, house)

#         # Append the score and the index to the list
#         scores.append((score, i))

#     # Sort the list by the score in descending order
#     scores = sorted(scores, reverse=True)

#     # Get the top n results
#     results = scores[:n]
#     dfs = []

#     # Print the results
#     print(f"Top {n} recommendations for the query house:")
#     for score, i in results:
#         # Get the house from the dataset
#         house = data[columns].iloc[i]

#         # Print the house features and the score
#         # print(f"House {i}:")
#         house_df = house.to_frame().T
#         house_df["Community"] = data["Community"]
#         house_df["ML_Number"] = data["ML_Number"]
#         # print(house.to_frame().T)
#         print(f"Similarity Score: {score:.4f}")
#         print()
#         dfs.append(house_df)

#     # Concatenate the DataFrames in the list
#     result_df = pd.concat(dfs)

#     return result_df


# query_house = df.iloc[0]
# nn_df = recommend(query_house, n=8)

# # all_recommendations = pd.DataFrame()

# # for i in range(len(df)):
# #     query_house = df.iloc[i]
# #     nn_df = recommend(query_house, n=8)
# #     all_recommendations = pd.concat([all_recommendations, nn_df])

# # # # Reset index of the resulting DataFrame
# # # all_recommendations.reset_index(drop=True, inplace=True)

# # # Display or further process the all_recommendations DataFrame as needed
# # all_recommendations.to_csv("NN.csv")
# nn_df.to_csv("NN.csv")
# print(query_house)
# nn_df

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import datetime

# Assuming df is your dataset containing house information, and 'sold_date' is the column representing the date when the house was sold.
# You might need to adjust the column names based on your actual dataset structure.


# Define the similarity metric
def similarity(house1, house2, date_range=30):
    # Calculate the cosine similarity between two houses
    cosine_sim = cosine_similarity(
        house1.drop("sold_date").values.reshape(1, -1),
        house2.drop("sold_date").values.reshape(1, -1),
    )[0][0]

    # Check if sold dates fall within the specified date range
    date_diff = abs((house1["Sold_Month"] - house2["Sold_Month"]).days)
    date_similarity = 1 / (1 + date_diff / date_range)

    # Combine cosine similarity and date similarity
    combined_similarity = (cosine_sim + date_similarity) / 2

    return combined_similarity


# Define the recommendation function
def recommend(query_house, n=10, date_range=30):
    # Initialize an empty list to store the similarity scores
    scores = []

    # Loop through all the houses in the dataset
    for i in range(len(df)):
        # Get the current house
        house = df.iloc[i]

        # Calculate the similarity score with the query house
        score = similarity(query_house, house, date_range)

        # Append the score and the index to the list
        scores.append((score, i))

    # Sort the list by the score in descending order
    scores = sorted(scores, reverse=True)

    # Get the top n results
    results = scores[:n]
    dfs = []

    # Print the results
    print(f"Top {n} recommendations for the query house:")
    for score, i in results:
        # Get the house from the dataset
        house = data[columns].iloc[i]

        # Print the house features and the score
        house_df = house.to_frame().T
        house_df["Community"] = data["Community"]
        house_df["ML_Number"] = data["ML_Number"]
        print(f"Similarity Score: {score:.4f}")
        print()
        dfs.append(house_df)

    # Concatenate the DataFrames in the list
    result_df = pd.concat(dfs)

    return result_df


# Example usage:
query_house = df.iloc[0]
nn_df = recommend(query_house, n=8, date_range=30)

# Save the result to a CSV file
nn_df.to_csv("NN.csv")

print(query_house)
print(nn_df)

KeyError: "['sold_date'] not found in axis"

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Assuming df is your original dataset and data is the dataset containing house features
# Replace 'columns' with the actual column names you want to consider in your similarity calculation


# Define the similarity metric for content-based filtering
def content_similarity(house1, house2):
    return cosine_similarity(
        house1.values.reshape(1, -1), house2.values.reshape(1, -1)
    )[0][0]


# Define the content-based recommendation function
def content_based_recommend(query_house, n=10):
    scores = []

    for i in range(len(df)):
        house = df.iloc[i]
        score = content_similarity(query_house, house)
        scores.append((score, i))

    scores = sorted(scores, reverse=True)
    results = scores[:n]

    return results


# Define the collaborative recommendation function (example: item-based collaborative filtering)
def collaborative_recommend(query_house, n=10):
    # Implement collaborative filtering logic based on user-item interactions
    # This could involve identifying houses that users with similar preferences have interacted with
    # For simplicity, let's assume collaborative_recommend returns a list of house indices

    # Replace this with your collaborative filtering logic
    collaborative_results = [
        (0.8, 0),
        (0.7, 1),
        (0.6, 2),
        (0.5, 3),
        (0.4, 4),
        (0.3, 5),
        (0.2, 6),
        (0.1, 7),
        (0.05, 8),
        (0.01, 9),
    ]

    return collaborative_results


# Define the hybrid recommendation function
def hybrid_recommend(query_house, n=10, content_weight=0.7, collaborative_weight=0.3):
    # Get content-based recommendations
    content_results = content_based_recommend(query_house, n=n)

    # Get collaborative recommendations
    collaborative_results = collaborative_recommend(query_house, n=n)

    # Combine the recommendations with weights
    hybrid_results = []
    for (content_score, content_index), (collab_score, collab_index) in zip(
        content_results, collaborative_results
    ):
        hybrid_score = (
            content_weight * content_score + collaborative_weight * collab_score
        )
        hybrid_results.append((hybrid_score, content_index))

    # Sort the hybrid results by score in descending order
    hybrid_results = sorted(hybrid_results, reverse=True)
    results = hybrid_results[:n]

    dfs = []

    # Print the results
    print(f"Top {n} recommendations for the query house:")
    for score, i in results:
        # Get the house from the dataset
        house = data[columns].iloc[i]

        # Print the house features and the score
        # print(f"House {i}:")
        house_df = house.to_frame().T
        house_df["Community"] = data["Community"]
        house_df["ML_Number"] = data["ML_Number"]
        # print(house.to_frame().T)
        print(f"Similarity Score: {score:.4f}")
        print()
        dfs.append(house_df)

    # Concatenate the DataFrames in the list
    result_df = pd.concat(dfs)

    return result_df


# Example usage:
query_house = df.iloc[0]  # Replace with your actual query house
hybrid_results = hybrid_recommend(query_house, n=10)

# Print or further process the hybrid_results as needed
hybrid_results.to_csv("NN1.csv")